# Lesson 10 神经网络的损失函数

我们将从梯度下降法向外拓展，介绍神经网络的损失函数、常用优化算法等信息，实现神经网络的学习和迭代。本节主要讲解神经网络常用的损失函数，并在PyTorch中实现这些函数。


##  一、机器学习中的优化思想

我们建立神经网络时总是先设定好$w$与$b$的值（或者由我们调用的PyTorch类帮助我们随机生成权重向量$w$），接着通过加和求出$z$，再在$z$上嵌套sigmoid或者softmax函数，最终获得神经网络的输出。我们的代码及计算流程，总是从神经网络的左侧向右侧计算的。

线性回归的任务就是构造一个预测函数来映射输入的特征矩阵$X$和标签值$y$线性关系。构造预测函数核心就是找出模型的权重向量$w$，并令线性回归的输出结果与真实值相近，也就是求解线性方程组中的$w$和$b$。对神经网络而言也是如此，我们的核心任务是求解一组最适合的$w$和$b$，**令神经网络的输出结果与真实值接近**。

**优化流程**
1. 提出基本模型，明确目标
2. 确定损失函数/目标函数
   - 转化为凸函数
   - 在凸函数上，求解对应L(w)最小值的w
3. 确定适合的优化算法
4. 确定优化算法，最小化损失函数，求解最佳权重

**损失函数**  
衡量真实值与预测结果的差异，评价模型学习过程中产生的损失的函数,以需要求解的权重向量$w$为自变量的函数$L(w)$ 。
  - 损失函数的值很小，则说明模型预测值与真实值很接近，模型在数据集上表现优异，权重优秀
  - 损失函数的值大，则说明模型预测值与真实值差异很大，模型在数据集上表现差劲，权重糟糕

## 二、回归: 误差平方和SSE

$$SSE= \sum_{i=1}^m{(z_i-\hat{z_i})^2}$$

- 均方误差：全部样本的平均损失：

$$MSE= \frac{1}{m}\sum_{i=1}^m{(z_i-\hat{z_i})^2}$$


- `criterion = MSELoss();criterion(预测，真实值)` 实例化MSELoss类
  - 参数：默认reduction = "mean"，返回均方误差
  - reduction = "sum",返回总体的误差平方和

In [2]:
import torch
from torch.nn import MSELoss  # MSELoss是一个类，调用需要实例化


In [3]:
# 随机生成数据
yhat = torch.randn(size=(50,),dtype=torch.float32)
y = torch.randn(size=(50,),dtype=torch.float32)


In [5]:
# 实例化
criterion = MSELoss()
loss = criterion(yhat,y)
loss


tensor(1.3172)

In [6]:
#————————————————————————————均方误差———————————————————————————

criterion = MSELoss(reduction = "mean") #实例化
criterion(yhat,y)

tensor(1.3172)

In [8]:
#————————————————————————————总体平方误差————————————————————————————
criterion = MSELoss(reduction = "sum")
criterion(yhat,y)


tensor(65.8581)

## 三、二分类交叉熵损失函数

我们将介绍二分类神经网络的损失函数：二分类交叉熵损失函数，也叫做对数损失。这个损失函数被广泛地使用在任何输出结果是二分类的神经网络中，即不止限于单层神经网络，还可被拓展到多分类中。

二分类交叉熵损失函数是由极大似然估计推导出来的，对于有m个样本的数据集而言，在全部样本上的平均损失写作：
$$L(w)=-\sum_{i=1}^m{(y_i*\ln{\sigma_i}+(1-y_i)*\ln{(1-\sigma_i}))}$$

- $\sigma_i$即在第1类上的概率
- 本来极大似然求得最大值，加-号变成求解损失函数的最小值
- 这个公式很重要！

### 1.极大似然估计求解二分类交叉熵损失

**极大似然估计**  
&emsp;&emsp;如果一个事件的发生概率很大，那这个事件应该很容易发生。相应的，如果依赖于权重 的任意事件的发生就是我们的目标，那我们只要寻找令其发生概率最大化的权重$w$就可以了。**寻找相应的权重$w$，使得目标事件的发生概率最大**.
其步骤如下：  
1. 构筑似然函数$P(w)$，用于评估目标事件发生的概率，该函数被设计成目标事件发生时，概率最大  
2. 对整体似然函数取对数，构成对数似然函数$\ln{P(w)}$
3. 在对数似然函数上对权重 求导，并使导数为0，对权重进行求解

### 2.用tensor实现二分类交叉熵损失

- 尽量使用torch中的运算符，运算更快

In [9]:
import torch
import time

In [23]:
# loss = -(y*ln(sigma)+(1-y)*ln(1-sigma))
# y 真实标签
# z = Xw
# X,w
# m个样本

N = 3 * pow(10,3)  # 3000个数据
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32) # 4个特征
w = torch.rand((4,1),dtype=torch.float32,requires_grad=True)
y = torch.randint(low=0,high=2,size=(N,1),dtype=torch.float32)   # 设置二分类的标签


In [24]:
# w是二维的，就是用torch.mm
zhat = torch.mm(X,w)
sigma = torch.sigmoid(zhat)
sigma.shape  # 返回3000个样本的概率 

torch.Size([3000, 1])

In [25]:
Loss = -(y*torch.log(sigma)+(1-y)*torch.log(1-sigma))  # 计算的是每个样本单独的损失 
Loss

tensor([[0.3075],
        [0.3073],
        [0.9198],
        ...,
        [0.3876],
        [0.4536],
        [0.3442]], grad_fn=<NegBackward0>)

In [26]:
# 返回均方误差
loss = 1/N*(-sum((y*torch.log(sigma)+(1-y)*torch.log(1-sigma))))
loss


tensor([0.7962], grad_fn=<MulBackward0>)

In [21]:
#————————————————————————————尽量使用Torch中的运算————————————————————————————
#你可以试着比较在样本量为300W时，以下两行代码运行的时间差异。这段代码不需要GPU。
#如果你的电脑内存或计算资源有限，可以试着将样本量调小为30W或3W

N = 3*pow(10,6)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,1),dtype=torch.float32,requires_grad=True)
y = torch.randint(low=0,high=2,size=(N,1),dtype=torch.float32)

zhat = torch.mm(X,w)
sigma = torch.sigmoid(zhat)

# torch.sum() 用了0.07秒
start = time.time()
L1 = -(1/N)*torch.sum((1-y)*torch.log(1-sigma)+y*torch.log(sigma))
now = time.time() #seconds
print(now - start)


# 计算的很慢...72秒
start = time.time()
L2 = -(1/N)*sum((1-y)*torch.log(1-sigma)+y*torch.log(sigma))
now = time.time() #seconds
print(now - start)

0.07133221626281738
72.7374906539917


从运行结果来看，除了加减乘除，我们应该尽量避免使用任何Python原生的计算方法。如果可能的话，让PyTorch处理一切。

### 3.用PyTorch中的类实现二分类交叉熵损失

**方法一：nn模块中的类**
- `class` BCEWithLogitsLoss 
  - 内置了sigmoid函数与交叉熵函数，它会自动计算输入值的sigmoid值
  - 因此需要输入**zhat与真实标签**，且顺序不能变化，zhat必须在前
  - 数据量较大时，计算sigmoid函数，可能有精度问题
- `class` BCELoss
  - 只有交叉熵函数，没有sigmoid层
  - 因此需要输入sigma与真实标签，且顺序不能变化
  - 需要监控准确率时使用
- 二分类交叉熵的类们也有`参数reduction`，
  - 默认是mean
  - sum 要求输出整体的损失
  - none 示不对损失结果做任何聚合运算，直接输出每个样本对应的损失矩阵
- 两个函数都要求预测值与真实标签的数据类型以及结构（shape）必须相同，否则运行就会报错。

**方法二：functional库中的计算函数**  
- 两个不常用的函数
- `function` F.binary_cross_entropy_with_logits
- `function` F.binary_cross_entropy


In [22]:
import torch.nn as nn


In [29]:
#————————————————————————————使用类：nn.BCELoss() ————————————————————————————
# 实例化（使用3000个数据的sigma）
criterion = nn.BCELoss() 
loss = criterion(sigma,y)
loss

tensor(0.7962, grad_fn=<BinaryCrossEntropyBackward0>)

In [28]:
#————————————————————————————使用类：nn.BCEWithLogitsLoss()————————————————————————————

criterion2 = nn.BCEWithLogitsLoss()  # 实例化
loss2 = criterion2(zhat,y)
loss2

#内置的sigmoid函数可以让精度问题被缩小（因为将指数运算包含在了内部），以维持算法运行时的稳定性，
# 即是说当数据量变大、数据本身也变大时，BCELoss类产生的结果可能有精度问题

tensor(0.7962, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [30]:
criterion2 = nn.BCEWithLogitsLoss(reduction = "mean")
loss = criterion2(zhat,y)
loss


tensor(0.7962, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [31]:
criterion2 = nn.BCEWithLogitsLoss(reduction = "sum")
loss = criterion2(zhat,y)
loss


tensor(2388.5840, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [32]:
criterion2 = nn.BCEWithLogitsLoss(reduction = "none")
loss = criterion2(zhat,y)
loss

tensor([[0.3075],
        [0.3073],
        [0.9198],
        ...,
        [0.3876],
        [0.4536],
        [0.3442]], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [33]:
#————————————————————————————方法二：使用函数————————————————————————————
from torch.nn import functional as F
#直接调用functional库中的计算函数
F.binary_cross_entropy_with_logits(zhat,y)


tensor(0.7962, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [34]:
F.binary_cross_entropy(sigma,y)

tensor(0.7962, grad_fn=<BinaryCrossEntropyBackward0>)

### 四、多分类交叉熵损失函数 

#### 1.由二分类推广到多分类

多分类问题中，不服从伯努利分布，样本标签被预测为k的概率为：
$$P_k=P(\hat{y}_i=k|x_i,w)=\sigma$$
在多分类问题中，$\sigma$就是softmax对于类别返回的值
- 对原来的真实标签[1,2,3],做独热编码[1,0,0],[0,1,0],[0,0,1]
- 交叉熵函数$$L(w)=-\sum_{i=1}^m{y_i(k=j)\ln{\sigma_i}}$$
  - 我们把$\ln{softmax(z)}$这样的函数单独定义了一个功能做logsoftmax，PyTorch中可以直接通过nn.logsoftmax类调用
  - 同时，我们把对数之外的，乘以标签、加和、取负等等过程打包起来，称之为负对数似然函数，在PyTorch中可以使用nn.NLLLoss来进行调用。
  - 也就是说，在计算损失函数时，我们不再需要使用单独的softmax函数了。

#### 2.用Pytorch实现多分类交叉熵损失

- 调用logsoftmax和MLLLoss实现
- 直接调用CrossEntropyLoss
  

In [1]:
import torch
import torch.nn as nn

N = 3*pow(10,2)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,3),dtype=torch.float32,requires_grad=True)
#定义y时应该怎么做？应该设置为矩阵吗？
y = torch.randint(low=0,high=3,size=(N,),dtype=torch.float32)


In [2]:
zhat = torch.mm(X,w)

In [6]:
#————————————————————————————里开始调用softmax和NLLLoss————————————————————————————
logsm = nn.LogSoftmax(dim=1)  # 实例化
logsigma = logsm(zhat)

criterion = nn.NLLLoss() # 实例化
criterion(logsigma,y.long())

# 必须把y转化为整形long

# 可以发现，两种输出方法得到的损失函数结果是一致的。

tensor(1.1591, grad_fn=<NllLossBackward0>)

- 直接调用CrossEntropyLoss

In [7]:
criterion_ = nn.CrossEntropyLoss() 
#对打包好的CorssEnrtopyLoss而言，只需要输入zhat
criterion_(zhat,y.long())

tensor(1.1591, grad_fn=<NllLossBackward0>)